In [ ]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Initialize()
ee.Authenticate

In [ ]:
shapefile_path = "C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp"

# Check if the path is a .zip file
if shapefile_path.endswith('.zip'):
    # Try to read shapefile from a zip archive
    try:
        # Check if the .zip file exists and open it
        with zipfile.ZipFile(shapefile_path, 'r') as zip_ref:
            zip_ref.printdir()  # Optional: Print contents of the zip to debug
            # Try to find the .shp file inside the zip
            shapefile_found = False
            for file in zip_ref.namelist():
                if file.endswith('.shp'):
                    shapefile_found = True
                    shapefile_within_zip = file
                    break

            if shapefile_found:
                # Read shapefile directly from the zip file
                oi = gpd.read_file(f'zip://{shapefile_path}/{shapefile_within_zip}')
                print(f"Successfully loaded shapefile from {shapefile_path}.")
            else:
                print("No .shp file found inside the zip archive.")
                #
    except Exception as e:
        print(f"Error reading shapefile from zip archive: {e}")
        #
else:
    # If not a .zip, assume it is a regular shapefile
    try:
        # Read the shapefile normally
        aoi = gpd.read_file(shapefile_path)
        print(f"Successfully loaded shapefile from {shapefile_path}.")
    except Exception as e:
        print(f"Error reading shapefile: {e}")


# After loading, check if the GeoDataFrame is not empty
if not aoi.empty:
    # If the GeoDataFrame contains multiple geometries, dissolve them into one
    if len(aoi) > 1:
        aoi = aoi.dissolve()

    # Extract the first geometry from the dissolved GeoDataFrame
    geometry = aoi.geometry.iloc[0]

    # Check if the geometry is a Polygon or MultiPolygon
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        # Convert the geometry to GeoJSON format
        geojson = geometry.__geo_interface__

        # Remove the third dimension from the coordinates if it exists
        if geojson['type'] == 'Polygon':
            geojson['coordinates'] = [list(map(lambda coord: coord[:2], ring)) for ring in geojson['coordinates']]
        elif geojson['type'] == 'MultiPolygon':
            geojson['coordinates'] = [[list(map(lambda coord: coord[:2], ring)) for ring in polygon] for polygon in geojson['coordinates']]

        # Create an Earth Engine geometry object from the GeoJSON coordinates
        ee_geometry = ee.Geometry(geojson)

        # Convert the Earth Engine geometry to a Feature
        feature = ee.Feature(ee_geometry)

        # Create a FeatureCollection with the feature
        aoi = ee.FeatureCollection([feature])

        print("AOI defined successfully.")

        # check_next_button()
    else:
        
        print("The geometry is not a valid type (Polygon or MultiPolygon).")
else:
    print("The shapefile does not contain any geometries.")        #

In [ ]:
# Define the start and end dates for filtering the image collection

import pandas as pd
from datetime import datetime
inicio = '2024-01-11'
final = '2025-01-11'
nuvem = 40

# Load the Sentinel-2 image collection and filter by date, location, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(inicio, final) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', nuvem)) \
    .map(lambda image: image.set('date', image.date().format('YYYY-MM-dd')))

# Get the number of images in the collection
count = sentinel2.size().getInfo()
print(f"Number of images in collection: {count}")

In [ ]:
# Get the original timestamps from the sentinel2 image collection
original_timestamps = sentinel2.aggregate_array('system:time_start').getInfo()

# Convert timestamps to formatted dates and times
formatted_dates_times = [datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d %H:%M') for ts in original_timestamps]

# Print the formatted dates and times
print(formatted_dates_times)

In [ ]:
# Step 1: Aggregate timestamps from the ImageCollection
original_timestamps = sentinel2.aggregate_array('system:time_start').getInfo()

# Step 2: Convert timestamps to formatted dates
formatted_dates = [datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in original_timestamps]

# Step 3: Identify unique dates and map them back to the original timestamps
df = pd.DataFrame(list(zip(original_timestamps, formatted_dates)), columns=['timestamp', 'date'])
first_timestamps_per_date = df.groupby('date')['timestamp'].min().tolist()

# Step 4: Filter the collection to include only the first image for each unique date
filtered_collection = sentinel2.filter(
    ee.Filter.inList('system:time_start', ee.List(first_timestamps_per_date))
)

# Step 5: Inspect the results
print(f"Number of first images (one per day): {filtered_collection.size().getInfo()}")
#print(filtered_collection.first().getInfo())

In [ ]:
filtered_collection.aggregate_array('date').getInfo()

In [ ]:
Date_list_selection = ['2024-01-14',
 '2024-01-24',
 '2024-02-05',
 '2024-02-08',
 '2024-02-23',
 '2024-02-25']

# Step 1: Filter the collection to include only images with dates in Date_list_selection
new_filtered_collection = filtered_collection.filter(
    ee.Filter.inList('date', Date_list_selection)
)

# Step 2: Inspect the results
print(f"Number of images matching Date_list_selection: {new_filtered_collection.size().getInfo()}")


In [ ]:
Date_list_selection = ['2024-01-24']

# Step 1: Filter the collection to include only images with dates in Date_list_selection
uniqueday_collection = filtered_collection.filter(
    ee.Filter.inList('date', Date_list_selection)
)

# Step 2: Inspect the results
print(f"Number of images matching Date_list_selection: {new_filtered_collection.size().getInfo()}")


In [103]:
first_image = uniqueday_collection.first()

In [104]:

# Clip image to AOI
first_image = first_image.clip(aoi)

# Get the acquisition date and define download region
region = aoi.geometry().bounds().getInfo()['coordinates']




In [105]:
index_image = first_image.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [106]:
# Generate download URL
try:
    url = first_image.getDownloadUrl({
        'scale': 10,
        'region': region,
        'format': 'GeoTIFF'
    })
except Exception as e:
    print(f"Failed to generate download URL: {e}")

In [ ]:
import requests

# Define the path to save the downloaded image
download_path = 'C:/Users/pc/Downloads/first_image_ndvi.tif'

# Download the image from the URL
try:
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(download_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Image successfully downloaded to {download_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
except Exception as e:
    print(f"An error occurred while downloading the image: {e}")